# Initiation & Functions

In [154]:
# import

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np
import math
import random

from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

import sys

In [155]:
# Membuat fungsi untuk menilai dampak policy terhadap lingkungan
# Semakin nilai fungsi ini mendekati 0 maka semuanya akan semakin bagus
def g(P, X):
    i = 0
    sigma_sx = 0
    sigma_s = 0
    while i < len(P):
        sigma_sx = sigma_sx + P[i][0]*X[i]
        sigma_s = sigma_s + P[i][0]
        i += 1
    return 1-(sigma_sx/sigma_s)



# Membuat fungsi payoff untuk action yang mungkin di timestep berikutnya
# untuk pemain-i.
# player adalah pemain ke berapa yang sedang dicari payoff functionnya (ingat indeks python)
# action adalah aksi (berapa persen (dalam desimal) investasi) pemain-i untuk timestep berikutnya
# P adalah matriks pemain. Nanti diambil aksi pemain lain selain pemain i
# X adalah matriks payoff
# Fungsi ini nanti digunakan untuk memilih max_payoff{0, 1, -b/2a} dengan syarat
# 0 <= -b/2a <= 1
def payoff(player, action, P, X):
    
    sigma_min_i = 0
    i = 0
    while i < len(P):
        if i != player:
            sigma_min_i = sigma_min_i + X[i]
        i += 1
    
    return (1 + alpha*(action+sigma_min_i)/N)*P[player][0]*P[player][1]*action + (1 - beta*(action+sigma_min_i)/N)*P[player][0]*P[player][2]*(1-action)



# membuat fungsi untuk menghitung -b/2a
# nanti jika 0 < -b/2a < 1 maka jadi calon titik optimum
# player adalah pemain ke berapa yang sedang dicari -b/2a nya
# P adalah matriks pemain. Nanti diambil aksi pemain lain selain pemain i
# X adalah matriks payoff
def min_b2a(player, P, X):
    
    sigma_min_i = 0
    i = 0
    while i < len(P):
        if i != player:
            sigma_min_i = sigma_min_i + X[i]
        i += 1
    
    return -(P[player][1]*P[player][0] - P[player][2]*P[player][0] + (alpha*P[player][1]*P[player][0]*sigma_min_i + beta*P[player][2]*P[player][0]*sigma_min_i - beta*P[player][2]*P[player][0])/N)/(2*(alpha*P[player][1]*P[player][0] + beta*P[player][2]*P[player][0])/N)



# mencari yang maksimal itu 0, 1, atau -b/2a
def x_optimum(player, P, X):
    
    # action
    a = 0
    b = min_b2a(player, P, X)
    #print(b)
    c = 1
    
    maksimal = -1
    
    # cari maksimal
    
    # saat 0 < b < 1 -> berarti b masuk
    if 0 < b < 1:
        if payoff(player, a, P, X) >= payoff(player, b, P, X) and payoff(player, a, P, X) >= payoff(player, c, P, X):
            maksimal = a
        if payoff(player, b, P, X) >= payoff(player, a, P, X) and payoff(player, b, P, X) >= payoff(player, c, P, X):
            maksimal = b
        if payoff(player, c, P, X) >= payoff(player, a, P, X) and payoff(player, c, P, X) >= payoff(player, b, P, X):
            maksimal = c
    else:
        if payoff(player, a, P, X) >= payoff(player, c, P, X):
            maksimal = a
        else:
            maksimal = c
    
    return maksimal

# Game Initiation

In [156]:
# mendefinisikan permainan

# pemain
N = 50                                 # N adalah banyak pemain

P = np.zeros((N,3))                   # P adalah state pemain
P = P.tolist()                        # ini tolist agar bentuknya pake list saja karena mudah

# inisiasi matriks P(S, A, B) dengan S adalah aset, A persentase keuntungan green investment, B persentase keuntungan biasa
# untuk random S, A, B ini nanti disesuaikan range nya
# dari ini yang penting kita asumsikan A << B menurut status quo

# randomisasi untuk update nilai A dan B
koefisien_random_A = 0.01
koefisien_random_B = 0.01


# state aksi yang dipilih di awal
X = []                                # X adalah matriks yang berisi aksi setiap pemain, isinya 0 <= xi <= 1

random.seed(0)                        # seed aja kalau main random inisiasi

# mengisi matriks permainan P
# dan matriks aksi X
i = 0
while i < N:
    P[i][0] = random.uniform(10, 20)
    hore = random.uniform(0, 5) # dalam persentase -> keunntungan ekonomi hijau
    P[i][1] = hore                # keuntungan ekonommi hijau
    P[i][2] = 10*hore        # dalam persentase. Asumsi penting: P[i][2] >> P[i][1] 
    
    # Matriks aksi dengan berbagai initial conditions
    
    X.append(random.uniform(0,1))
    #X.append(0)
    #X.append(1)
    
    i += 1

# inisiasi alpha dan beta sebagai faktor koreksi insentif dan pajak
# untuk nilainya nanti kita kuli satu satu dan jika N(+1) pemain nanti
# ditentukan melalui payoff untuk pemerintah
alpha = 1       # koefisien insentif ekonomi hijau
beta  = 10       # koefisien pajak ekonomi tidak hijau
threshold = 1 # koefisien keuntungan minimum yang harus dicapai produsen

# memasukkan time_step maksimal untuk penelitian
# nanti time mulai dari 1 .. time_step karena untuk 0 udah dipake di list E
# Ini BANYAKNYA RONDE PERMAINAN
max_round = 10

# Membuat list penampung, E
# list E dan F berisi nilai emisi dari state awal permainan sampai akhir
# semakin nilai elemen list E mendekati 0 maka semakin kecil emisi ke lingkungan
# dari formulanya, 0 <= g(P) <= 1
# memasukkan nilai E dan F di kondisi awal
E = []
E.append(g(P, X))

# Game

In [157]:
# Keperluan plotting pemain
P_plot = np.zeros((N,max_round+1))                   # P_plot adalah container yang berisi uang pemain per time step
P_plot = P_plot.tolist()

# Plot Actions -> aksi pemain i pada waktu j
plot_actions = np.zeros((N, max_round+1))
plot_actions = plot_actions.tolist()

i = 0
while i < N:
    P_plot[i][0] = P[i][0]
    plot_actions[i][0] = X[i]
    i += 1

    
time_control = 1

# PERMAINAN LOOP
while time_control <= max_round:
    
    # Linear Programming to update the value of alpha and beta (govt action)
    
    # Create the model
    model = LpProblem(name="govt", sense=LpMaximize)
    
    # Initialize the decision variables
    a = LpVariable(name="a", lowBound=0)
    b = LpVariable(name="b", lowBound=0)
    
    # Define objective functions
    j = 0
    sum_j = 0
    while j < N:
        sum_j = sum_j + X[j]
        j = j + 1
    
    i = 0
    sum_i_beta = 0
    sum_i_alpha = 0
    while i < N:
        sum_i_beta = sum_i_beta + (sum_j*P[i][2]*P[i][0]*(1-X[i])/N)
        sum_i_alpha = sum_i_alpha + (sum_j*P[i][1]*P[i][0]*X[i]/N)
        i = i + 1
    
    print("sum_beta = ", sum_i_beta)
    model += b*sum_i_beta - a*sum_i_alpha # objective funciton
    
    # Add the constraints to the model
    i = 0
    while i < N:
        model += (sum_j/N)*(a*P[i][1]*P[i][0]*X[i] - b*P[i][2]*P[i][0]*(1-X[i])) >= ((1+threshold)*P[i][0] - P[i][1]*P[i][0]*X[i] - P[i][2]*P[i][0]*(1-X[i]))
        i += 1
        
    model += b*sum_i_beta - a*sum_i_alpha >= 0 # pemerintah ngga boleh rugi
        
    # Solving
    model.solve()
    
    # Optimum alpha and beta
    for v in model.variables():
        print(time_control, " --> ", v.name, "=", v.varValue)
        print(model.variables,"\n")
        if v.varValue !=  None:
            if v.name == 'a':
                alpha = v.varValue
            else:
                beta = v.varValue
    
    
    # update nilai X (player action)
    i = 0
    while i < N:
        X[i] = x_optimum(i, P, X)
        i += 1
    
    # update matrix P berdasarkan X untuk tiap round
    i = 0
    while i < N:
        
        # keuntungan total (uang total enaknya)
        P[i][0] = P[i][0] + payoff(i, X[i], P, X) # ini pakai payoff instead of P[i][0] += x[i]*A[i] + (1-x[i])*B[i]
                                                  # karena kalau pakai P[i][0] += x[i]*A[i] + (1-x[i])*B[i] artinya
                                                  # pajak dan insentif belum masuk ke perhitungan keuntungan
        
        # randomisasi persentase keuntungan dari ekonomi hijau dan non hijau
        P[i][1] = P[i][1] + random.uniform(-koefisien_random_A, koefisien_random_A)*P[i][1]      # dalam persentase
        P[i][2] = P[i][2] + random.uniform(-koefisien_random_B, koefisien_random_B)*P[i][2]      # dalam persentase
        
        # update P_plot
        P_plot[i][time_control] = P[i][0]
        # update plot action
        plot_actions[i][time_control] = X[i]
        
        i += 1
        
    # Melihat kondisi lingkungan
    E.append(g(P, X))
    
    time_control += 1

sum_beta =  4783.597699504086
1  -->  a = 104.78171
<bound method LpProblem.variables of govt:
MAXIMIZE
-590.3384701339006*a + 4783.597699504086*b + 0.0
SUBJECT TO
_C1: 15.3807652615 a - 211.903345524 b >= -397.52614791

_C2: 6.81821051234 a - 100.196154 b >= -179.361263391

_C3: 6.74562808825 a - 74.0810921426 b >= -118.810437321

_C4: 18.9837597309 a - 186.311682429 b >= -360.719376446

_C5: 15.672012959 a - 96.7209879831 b >= -189.183182287

_C6: 29.2482298696 a - 5.12313588491 b >= -40.6848420664

_C7: 13.2501316028 a - 294.718914389 b >= -552.425651573

_C8: 27.8205690663 a - 128.537330162 b >= -264.254664673

_C9: 1.68376285612 a - 21.9433958772 b >= -15.7164061381

_C10: 37.1899366849 a - 12.8481159414 b >= -63.4214136058

_C11: 8.7093279927 a - 247.247798303 b >= -459.677939418

_C12: 0.363808371437 a - 255.453313472 b >= -452.849983919

_C13: 14.7993171632 a - 31.4262050171 b >= -53.9580614764

_C14: 0.0246152280409 a - 0.252557846469 b >= 32.8332952445

_C15: 3.87531245103 a 

3  -->  a = 351.53305
<bound method LpProblem.variables of govt:
MAXIMIZE
-1052674857.9259467*a + 0.0
SUBJECT TO
_C1: 37796217.0942 a >= -17990115.0308

_C2: 7778376.58729 a >= -1692973.29703

_C3: 2380892.30846 a >= 712063.368576

_C4: 54914552.2531 a >= -30725439.7051

_C5: 25793295.1453 a >= -12169079.9396

_C6: 44126523.2714 a >= -24868927.3887

_C7: 61880232.8691 a >= -34411734.722

_C8: 57689800.623 a >= -31768105.5641

_C9: 71748.6650221 a >= 214143.898895

_C10: 57010301.5494 a >= -31980312.5691

_C11: 45175135.3481 a >= -24483214.6891

_C12: 13684909.234 a >= -3641391.53799

_C13: 5008482.61434 a >= 75194.8501712

_C14: 0.45552549992 a >= 160.132270003

_C15: 1264758.4703 a >= 813215.883567

_C16: 616443.030715 a >= 671205.827939

_C17: 52183193.6788 a >= -30506431.4226

_C18: 35960.8172801 a >= 143070.86675

_C19: 55729867.1272 a >= -31545372.0808

_C20: 25276314.8126 a >= -10894756.7143

_C21: 13526804.235 a >= -3620618.21822

_C22: 15495880.7292 a >= -5097095.06608

_C23: 4

ZeroDivisionError: float division by zero

# Plotting for Analysis

In [ ]:
# making the space
sumbu_x = []
i = 0
while i <= max_round:
    sumbu_x.append(i)
    i += 1

In [ ]:
# plot kondisi lingkungan
plt.figure(figsize=(20,6))
plt.scatter(sumbu_x, E)
plt.title("Plot of Environmental Harm Indicator on Every Time Steps")
plt.xlabel("Time")
plt.ylabel("Environmental Harm Indicator")
plt.grid(True)
plt.ylim(-0.2, 1.2)
plt.show()

In [ ]:
# Plotting Kondisi Setiap Pemain

plt.figure(figsize=(20,6))
i = 0
while i < N:
    plt.plot(sumbu_x, P_plot[i])
    i += 1
plt.title("Plot of Player's Assets on Every Time Steps")
plt.xlabel("Time")
plt.ylabel("Player's Assets")

plt.show()

In [ ]:
# Plotting Actions per Rounds

plt.figure(figsize=(20,6))
i = 0
while i < N:
    plt.plot(sumbu_x, plot_actions[i], marker = 'o')
    i += 1
plt.title("Plot of Every Player's Action on Every Time Steps")
plt.xlabel("Time")
plt.ylim(-0.2, 1.2)
plt.ylabel("Action (Percentage in Decimal of Investment on Green Economy)")

plt.show()

In [ ]:
# Checking Numerical Nash
dummy = np.array(plot_actions).T.tolist() # transpose
#print("Player actions: \n", dummy, "\n")
nash = False
i = 0
while i < len(dummy)-1 and nash == False:
    if dummy[i] == dummy[i + 1]:
        nash = True
    i = i + 1
print("Nash exist: ", nash)

In [ ]:
print(model.variables)

Note:
Menggunakan linear programming tidak membantu untuk menentukan nilai alpha dan beta yang bagus untuk payoff permainan. Adapun alasannya adalah nilai beta akan terus dipaksa menjadi 0 dalam linear programming dan ada error untuk menentukan solusinya, yakni solusi yang dianggap optimal biasanya tidak memenuhi minimal salah satu constraint yang diberikan. Trik selanjutnya adalah dengan mencoba memberikan diskon terhadap payoff.